<a href="https://colab.research.google.com/github/SunbirdAI/sunbird-ai-api/blob/add-tts/Sunbird_API_sample_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#@title Setup
import requests
from IPython.display import clear_output

url = 'https://sunbird-ai-api-5bq6okiwgq-ew.a.run.app'

In [14]:
#@title Authentication

#@markdown Use your token or your credentials. If you choose `auth_type = token`, enter your access token, otherwise enter your credentials:

auth_type = 'login-credentials'  #@param ["token", "login-credentials"]

if auth_type == 'token':
  token = input('Enter your authentication token: ')
else:
  username = input('Enter your username: ')
  password = input('Enter your password: ')
  creds = {
      'username': username,
      'password': password
  }
  response = requests.post(f'{url}/auth/token', data=creds)
  token = response.json()['access_token']

clear_output()

In [44]:
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

In [18]:
#@title Translation from English
payload = {
    "source_language": "English",
    "target_language": "Luganda",
    "text": "Hello, how are you?"
}

response = requests.post(f"{url}/tasks/translate", headers=headers, json=payload)

if response.status_code == 200:
  translated_text = response.json()["text"]
  print("Translated text: ", translated_text)
else:
  print("Error: ", response.status_code, response.text)

Translated text:  Oli otya?


In [25]:
#@title Translation to English
payload = {
    "source_language": "Runyankole",
    "target_language": "English",
    "text": "Oburwaire bwa Korona ku bwatandikire, abantu baatandika kukora masiki z'emyenda."
}

response = requests.post(f"{url}/tasks/translate", headers=headers, json=payload)

if response.status_code == 200:
  translated_text = response.json()["text"]
  print("Translated text: ", translated_text)
else:
  print("Error: ", response.status_code, response.text)

Translated text:  When the coronavirus started, people started making masks.


In [61]:
#@title Batch translation
payload = {
    "requests": [
        {
          "source_language": "Runyankole",
          "target_language": "English",
          "text": "Oburwaire bwa Korona ku bwatandikire, abantu baatandika kukora masiki z'emyenda."
        },
        {
          "source_language": "Acholi",
          "target_language": "English",
          "text": "I kare ma two Corona opoto, dano ocako yubu bongi me boyo um."
        },
        {
          "source_language": "English",
          "target_language": "Luganda",
          "text": "This translation service is really cool."
        }
    ]
}

response = requests.post(f"{url}/tasks/translate-batch", headers=headers, json=payload)

if response.status_code == 200:
  translated_text = response.json()
  print("Response: ", translated_text)
else:
  print("Error: ", response.status_code, response.text)

Response:  {'responses': [{'text': 'When the coronavirus started, people started making masks.', 'confidences': None}, {'text': 'During the coronavirus pandemic, people started making sanitary pads.', 'confidences': None}, {'text': "Empeereza eno ey'okuvvuunula eba nnyuvu nnyo.", 'confidences': None}]}


In [47]:
#@title Speech to text

payload = {}
files=[
  ('audio', ('file.wav', open('LUGANDA+980.wav','rb'),'audio/wav'))  # upload your file to the colab environment and change it here in open
]
headers_stt = {
  'Authorization': f'Bearer {token}'
}

response = requests.request("POST", f'{url}/tasks/stt', headers=headers_stt, data=payload, files=files)

print(response.text)

{"text":"baminisita abaliko baawangulwa mu kulonda kwebibiina","confidences":null}


In [59]:
#@title Text to Speech
import base64

payload = {
    "text": "Oli otya?"
}

response = requests.post(f"{url}/tasks/tts", headers=headers, json=payload)

if response.status_code == 200:
  base64_string = response.json()["base64_string"]

  with open("temp.wav", "wb") as wav_file:
    decoded_audio = base64.decodebytes(base64_string.encode('utf-8'))
    wav_file.write(decoded_audio)
    wav_file.close()
else:
  print("Error: ", response.status_code, response.text)

In [60]:
# confirm that the audio has been created
from IPython.display import Audio
from IPython.display import display

display(Audio("temp.wav"))